In [8]:
import os
import sys
import pymeshlab
conda_active_path = sys.exec_prefix
# conda_active_path = os.environ["CONDA_PREFIX"]
sys.path.append(os.path.join(conda_active_path, "lib"))
import fileinput
import numpy as np

In [9]:
# freecad Macro - create shape and calculate deformations and save ply
import FreeCAD
import Part
import ObjectsFem
import Fem
import Mesh
import femmesh.femmesh2mesh
try:
    from femtools import ccxtools
except ImportError:
    print('ccxtools_import_passed...')
from femtools import ccxtools


In [10]:

doc = App.newDocument("doc")

box_obj = doc.addObject('Part::Box', 'Box')
box_obj.Height = 2
box_obj.Width = 5
box_obj.Length = 50

analysis_object = ObjectsFem.makeAnalysis(FreeCAD.ActiveDocument, 'Analysis')

femmesh_obj = doc.addObject('Fem::FemMeshShapeNetgenObject', 'CubeFemMesh')
femmesh_obj.Shape = box_obj 
femmesh_obj.Fineness = "VeryCoarse"
femmesh_obj.MaxSize = 100

doc.Analysis.addObject(femmesh_obj)
doc.recompute()

doc.addObject("Fem::ConstraintFixed","FemConstraintFixed")
doc.FemConstraintFixed.Scale = 1
doc.Analysis.addObject(doc.FemConstraintFixed)

doc.recompute()

doc.FemConstraintFixed.Scale = 1
doc.FemConstraintFixed.References = [(doc.Box,"Face2")]
doc.recompute()

doc.addObject("Fem::ConstraintForce","FemConstraintForce")
doc.FemConstraintForce.Force = 1.0
doc.FemConstraintForce.Reversed = False
doc.FemConstraintForce.Scale = 1
doc.Analysis.addObject(doc.FemConstraintForce)

doc.recompute()
doc.FemConstraintForce.Force = 1000
doc.FemConstraintForce.Direction = None
doc.FemConstraintForce.Reversed = True
doc.FemConstraintForce.Scale = 1
doc.FemConstraintForce.References = [(doc.Box,"Face6")]
doc.recompute()

analysis_object.addObject(ObjectsFem.makeSolverCalculixCcxTools(doc))

## material
material_object = ObjectsFem.makeMaterialSolid(doc, "SolidMaterial")
mat = material_object.Material
mat['Name'] = "Steel-Generic"
mat['YoungsModulus'] = "210000 MPa"
mat['PoissonRatio'] = "0.30"
mat['Density'] = "7900 kg/m^3"
material_object.Material = mat
analysis_object.addObject(material_object)

# run the analysis step by step
fea1 = ccxtools.FemToolsCcx()
fea1.update_objects()
fea1.setup_working_dir('/scratch/aifa/MyRepo/RL_FEM/gym_RLFEM/FreeCAD_RL_FEM_Environment/my_inp_files')

fea1.setup_ccx()
message = fea1.check_prerequisites()
if not message:
    fea1.purge_results()
    fea1.write_inp_file()
    fea1.ccx_run()
    print(f'+o+o+o+o+o+o+ .inp filename {fea1.inp_file_name}')
    fea1.load_results()
else:
    FreeCAD.Console.PrintError("Oh, we have a problem! {}\n".format(message))  # in report view
    print("Oh, we have a problem! {}\n".format(message))  # in python console

femmesh_obj = doc.getObject("ResultMesh").FemMesh
result = doc.getObject("CCX_Results")

out_mesh = femmesh.femmesh2mesh.femmesh_2_mesh(femmesh_obj, result)

for o in doc.Objects:
    print(o.Name)

+o+o+o+o+o+o+ .inp filename /scratch/aifa/MyRepo/RL_FEM/gym_RLFEM/FreeCAD_RL_FEM_Environment/my_inp_files/CubeFemMesh.inp
found a last Face:  167125904785538
CCX_Results
Mesh by surface search method:  0.3803160000000001
Box
Analysis
CubeFemMesh
FemConstraintFixed
FemConstraintForce
CalculiXccxTools
SolidMaterial
ResultMesh
CCX_Results


In [11]:
# calculate  displacements and add updated femmesh to the analysis (remove the old one)
mesh_obj = doc.CubeFemMesh
#remove the dold femmesh
doc.removeObject('CubeFemMesh')
#add updated femmesh - with the same name (->>> update old femmesh)
mesh_obj = doc.addObject('Fem::FemMeshObject', 'CubeFemMesh')

new_mesh = Fem.FemMesh()
res_coord = doc.CCX_Results.DisplacementVectors
res_nd_no = doc.CCX_Results.NodeNumbers
res_mesh = doc.ResultMesh.FemMesh
for n in res_nd_no:
    new_mesh.addNode((res_mesh.Nodes[n].x + res_coord[n-1].x), (res_mesh.Nodes[n].y + res_coord[n-1].y), (res_mesh.Nodes[n].z + res_coord[n-1].z), n)

for v in res_mesh.Volumes:
    new_mesh.addVolume( list(res_mesh.getElementNodes(v)))

mesh_obj.FemMesh = new_mesh
doc.Analysis.addObject(mesh_obj)

[<Fem::FemMeshObject object>]

In [12]:
for o in doc.Objects:
    print(o.Name)

Box
Analysis
FemConstraintFixed
FemConstraintForce
CalculiXccxTools
SolidMaterial
ResultMesh
CCX_Results
CubeFemMesh


In [13]:
# update the last inp file (nodes and elements): 
fea2 = ccxtools.FemToolsCcx()#
fea2.update_objects()#

fea2.inp_file_name = fea1.inp_file_name#
wd = fea2.inp_file_name.replace('CubeFemMesh.inp','')
fea2.working_dir = wd
message = fea2.check_prerequisites()#
print(message)

print(f'+o+o+o+o+o+o+ .inp filename {fea2.inp_file_name}')
if not message:
    fea2.reset_all()#
    fea2.ccx_run()
    fea2.load_results()#
    print('ccx run done!')
else:
    print("Oh, we have a problem! {}\n".format(message))  # in python console

print(f'+o+o+o+o+o+o+ .inp filename {fea2.inp_file_name}')


+o+o+o+o+o+o+ .inp filename /scratch/aifa/MyRepo/RL_FEM/gym_RLFEM/FreeCAD_RL_FEM_Environment/my_inp_files/CubeFemMesh.inp
ccx run done!
+o+o+o+o+o+o+ .inp filename /scratch/aifa/MyRepo/RL_FEM/gym_RLFEM/FreeCAD_RL_FEM_Environment/my_inp_files/CubeFemMesh.inp


In [14]:
inp_dir, inpfile = os.path.split(fea2.inp_file_name)
inp_dir

'/scratch/aifa/MyRepo/RL_FEM/gym_RLFEM/FreeCAD_RL_FEM_Environment/my_inp_files'